In [1]:
# Follow the instructions on https://github.com/sethmlarson/virtualbox-python
# regarding installation of the VirtualBox SDK. If in virtual environment,
# make sure to activate it before installing the SDK.
#
# VirtualBox's installation script (which installs vboxapi) tends to error
# out when I do it, but it seems to work anyways.

import virtualbox


In [4]:

# Note that VirtualBox (the GUI) doesn't need to be open to run this.
vbox = virtualbox.VirtualBox()
for machine in vbox.machines:
    print(machine.name)
    print(machine.state)
    print(type(machine.state))
    print(machine.memory_size)
    print(machine.id_p)
    

ubuntu2404
PoweredOff
<class 'virtualbox.library.MachineState'>
8192
9e896d5d-038f-423f-a420-594ba8c41643
Windows 11
PoweredOff
<class 'virtualbox.library.MachineState'>
4096
5ed50965-0440-4424-a4ce-1576a6369bd1
win11-nounattend
PoweredOff
<class 'virtualbox.library.MachineState'>
16384
20d34a1d-0226-4827-92ee-03274c86f66f


In [11]:
# Launch the VM. The second argument to launch_vm_process, `name`, dictates
# the frontend that the GUI will use. The three usable ones are:
# - "gui": VirtualBox Qt GUI front-end
# - "headless": VBoxHeadless (VRDE Server) front-end
# - "sdl": VirtualBox SDL front-end
#
# "gui" is the standard GUI frontend (e.g. the one you see when you start a VM
# normally from the VBox GUI) based on Qt. "sdl" is an alternative 
# SDL-based GUI frontend, which is lightweight and generally Linux-only.
# (see https://forums.virtualbox.org/viewtopic.php?t=104104)

session = virtualbox.Session()
machine = vbox.find_machine("win11-nounattend")

# Returns an async `IProgress` object. wait_for_completion() is basically the
# same as awaiting that object.
#
# Additionally, this locks the machine to the provided session. Opening a GUI
# instance implicitly creates a lock on the machine.
progress = machine.launch_vm_process(session, "gui", [])
progress.wait_for_completion()

In [2]:
# Bind a new session to a machine, typically an already-running VM.
#
# There are three lock types: shared (1), write (2), and vm (3).
# Shared locks allow multiple distinct sessions to operate on the same machine
# at once, with the caveat that various operations -- such as changing the 
# machine state or creating snapshots - are disallowed. The write lock may
# only be held by exactly one session at a time.

import virtualbox
vbox = virtualbox.VirtualBox()

session = virtualbox.Session()
machine = vbox.find_machine("win11-nounattend")
machine.lock_machine(session, virtualbox.library.LockType(1))

# Verify that this machine is at the desktop. The `additions_run_level` can
# be one of three values:
# - 0: Guest Additions are not loaded
# - 1: Guest drivers are loaded
# - 2: Common components (such as application services) are loaded.
# - 3: Per-user desktop components are loaded.
#
# This is described as part of `AdditionsRunLevelType`.
import time
if session.console.guest.additions_run_level != virtualbox.library.AdditionsRunLevelType.desktop:
    print("Not at desktop yet.")
    time.sleep(1)
print("At desktop.")

AttributeError: Failed to find attribute guest in <virtualbox.library_ext.console.IConsole object at 0x00000207388FB790>

In [ ]:
# We can check various strings to figure out which one is the host-only interface.
# The "HostOnly" string enumeration member (from the NetworkAdapterType enum) is
# what we're looking for.
print(machine.get_network_adapter(0).attachment_type) # NAT == 1
print(machine.get_network_adapter(1).attachment_type) # HostOnly == 4

# With the Guest Additions installed, we can get the IP address of the guest.
# Because we've configured eth1 (adapter 2) as the host-only adapter, we use
# .../Net/1/V4/IP to get the IP address of the guest over the maintanance
# interface. Consistent with the `VBoxManage guestproperty enumerate` command,
# a set of tuples - here, column-major order.
#
# Note that the host-only adapter should be configured to be the *second* adapter
# in VirtualBox. This is because *plenty* of things will simply go slower on Windows
# when it tries to use the host-only adapter as the primary network interface,
# and since it doesn't have internet, things like SmartScreen and Defender
# checking downloaded files will take forever.
result = machine.enumerate_guest_properties("/VirtualBox/GuestInfo/Net/1/V4/IP")
host_ip_address = result[1][0]
print(host_ip_address)

# The IP address of the virtual host-only adapter for the *host* is 
# 192.168.56.1/24 by default.

In [1]:
# To get a network capture *at the hypervisor level*, you need to configure
# the desired interface's "trace file". At a lower level, that means you need
# to enable tracing for that interface, as well as configure the output path
# for the trace file.
#
# reference: synthesizers-combined/pyvmpop/pyvmpop/hypervisor/hv_vbox.py:1635

import virtualbox

# Get our NAT interface
vbox = virtualbox.VirtualBox()
session = virtualbox.Session()
machine = vbox.find_machine("win11-nounattend")

# progress = machine.launch_vm_process(session, "gui", [])
# progress.wait_for_completion()

# Need a lock to change these settings. A shared lock is sufficient.
# This doesn't happen in VMPOP's hv_vbox.py because they already have a 
# session lock, likely as a result of starting the machine.
machine.lock_machine(session, virtualbox.library.LockType.shared)

# Getting the network adapter *has* to be bound to a session for this to work.
# Without a session holding a lock, you'll get inexplicable errors about needing
# the machine to be saved/running... when the machine is already in those states.
network_adapter = session.machine.get_network_adapter(0)

# Power off the machine if it is currently on
# if machine.state == virtualbox.library.MachineState(5):
#     # Guest Additions shutdown, i.e. a normal ACPI shutdown.
#     # session.console.guest.shutdown()
    
#     # Note that this is a VM shutdown, not an ACPI shutdown.    
#     progress = session.console.power_down()
#     progress.wait_for_completion()

# You're supposed to only configure tracing when the machine is off, but that
# doesn't seem to be a hard requirement.
# https://forums.virtualbox.org/viewtopic.php?t=93612
network_adapter.trace_enabled = True
network_adapter.trace_file = ""
session.machine.save_settings()
session.unlock_machine()